In [6]:
!pip install ultralytics datasets huggingface-hub transformers fiftyone decord av -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [7]:
from fiftyone.utils.kinetics import Kinetics7002020DatasetInfo
from fiftyone.utils.kinetics import KineticsDatasetDownloader
import tarfile
import os
import av
import numpy as np
import numpy
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
from ultralytics import YOLO
import cv2
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

Migrating database to v0.23.8


INFO:fiftyone.migrations.runner:Migrating database to v0.23.8


# Download kinetics dancing

In [8]:
kinetics_dir = "."
scratch_dir = '.'
split = "train"

dataset_info = Kinetics7002020DatasetInfo(kinetics_dir, scratch_dir, split)

 100% |████|  189.7Mb/189.7Mb [242.5ms elapsed, 0s remaining, 782.5Mb/s]      


INFO:eta.core.utils: 100% |████|  189.7Mb/189.7Mb [242.5ms elapsed, 0s remaining, 782.5Mb/s]      


 100% |████|  388.3Kb/388.3Kb [26.4ms elapsed, 0s remaining, 24.5Mb/s] 


INFO:eta.core.utils: 100% |████|  388.3Kb/388.3Kb [26.4ms elapsed, 0s remaining, 24.5Mb/s] 


In [9]:
dancing_classes = [cls for cls in dataset_info.all_classes if "dancing" in cls]
dancing_classes = dancing_classes[:2]

In [ ]:
downloader = KineticsDatasetDownloader(num_workers=None)

dataset_dir = "."

split = "train"

did_download = False
for cls in dancing_classes:
    try:
        downloader.download_classes(dataset_info, [cls])
        did_download = True
    except KeyError:
        print(f"Skipping class '{cls}' as it is not found in the dataset")

if did_download:
    print("Downloaded the dataset with the tag 'dancing'")
    classes = dancing_classes
    print("Classes:", classes)
else:
    print("No classes with the tag 'dancing' found in the dataset")


INFO:fiftyone.utils.kinetics:Downloading 1 tars...


 100% |██████|    5.6Gb/5.6Gb [48.5s elapsed, 0s remaining, 127.3Mb/s]      


INFO:eta.core.utils: 100% |██████|    5.6Gb/5.6Gb [48.5s elapsed, 0s remaining, 127.3Mb/s]      


INFO:fiftyone.utils.kinetics:Downloading 1 tars...


 100% |██████|    7.0Gb/7.0Gb [59.8s elapsed, 0s remaining, 115.0Mb/s]      


INFO:eta.core.utils: 100% |██████|    7.0Gb/7.0Gb [59.8s elapsed, 0s remaining, 115.0Mb/s]      


Downloaded the dataset with the tag 'dancing'
Number of samples: None
Classes: ['belly dancing', 'breakdancing']


In [ ]:
os.chdir("./")

for filename in os.listdir():
    if filename.startswith("k700") and filename.endswith(".tar.gz"):
        with tarfile.open(filename, "r:gz") as tar:
            tar.extractall("./kinetics")


In [ ]:
os.chdir("./")

files = os.listdir()

tar_gz_files = [f for f in files if f.endswith(".tar.gz")]

for f in tar_gz_files:
    os.remove(f)


# Dataset class

In [ ]:
dataset_dir = "./kinetics"

class KineticsDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_dir, transform=None):
        self.dataset_dir = dataset_dir
        self.classes = sorted(os.listdir(dataset_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        self.samples = []
        for cls in self.classes:
            class_dir = os.path.join(dataset_dir, cls)
            for video_file in os.listdir(class_dir):
                self.samples.append((os.path.join(class_dir, video_file), self.class_to_idx[cls]))
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]
        return video_path, label


# Prepare dataset

In [ ]:
yolov8 = YOLO('yolov8n-pose.pt')

all_tensors = []

for folder in tqdm(os.listdir('./kinetics')):
    path = f"./kinetics/{folder}/"
    for video_path in tqdm(os.listdir(path)):
        all_tensors.append([])
        cap = cv2.VideoCapture(path + video_path)
        frame_count = 0
        while cap.isOpened() and frame_count<30:
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            results = yolov8(frame, verbose=False)
            if len(results)==0:
              continue
            results = results[0].keypoints.data[0]
            all_tensors[len(all_tensors)-1].append(results)
        cap.release()

100%|██████████| 2/2 [2:41:01<00:00, 4830.99s/it]


In [ ]:
new_tensors = []
for tensor_list in all_tensors:
  new_tensors.append([])
  for tensor in tensor_list:
    if tensor.shape[0] != 0 and tensor.numel()!=0:
      new_tensors[len(new_tensors)-1].append(tensor)

In [ ]:
new_tensors

In [ ]:
stacked_tensors = []
for tensor_list in new_tensors:
  if len(tensor_list)==0:
    continue
  combined_tensor = torch.stack(tensor_list, dim=0)
  stacked_tensors.append(combined_tensor)

In [ ]:
dfs = []

for i in range(len(stacked_tensors)):
    print(i)
    df = pd.DataFrame({'tensor': [stacked_tensors[i].tolist()], 'class_name': [dancing_classes[i // int((len(stacked_tensors)+1)/2)]]})
    dfs.append(df)

result_df = pd.concat(dfs, ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
len(stacked_tensors)

1741

In [ ]:
result_df.to_csv("result.csv")

In [1]:
result_df = pd.read_csv("result (1).csv")

# Define RNN

In [2]:
class PoseRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1):
        super(PoseRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.rnn(x, h0)

        out = torch.mean(out, dim=1)

        out = out.view(out.size(0), -1)

        out = self.fc(out)

        out = F.softmax(out, dim=1)
        return out


input_size = 17*3  # number of features
hidden_size = 128  # number of features in hidden state
num_layers = 2  # number of stacked RNN layers
num_classes = 2  # number of output classes

model = PoseRNN(input_size, hidden_size, num_layers, num_classes)


# Train loop

## Train-test split

In [3]:
df = pd.read_csv('result (1).csv')

def string_to_tensor(s):
  return torch.tensor(eval(s))

df['tensor'] = df['tensor'].apply(string_to_tensor)

X = df['tensor'].tolist()
y = df['class_name'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Main loop

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

label_to_index = {class_name: index for index, class_name in enumerate(dancing_classes)}

y_train_indices = [label_to_index[label] for label in y_train]

num_epochs = 10
running_loss = 0
for epoch in range(num_epochs):
    for i, data in enumerate(X_train):
        inputs, labels = data, y_train_indices[i]
        data = data.view(data.shape[0],-1).unsqueeze(0)
        
        outputs = model(data)
        loss = criterion(outputs, torch.tensor(labels).unsqueeze(0))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
    print(running_loss/(epoch+1))

924.0412410795689
905.9623271971941
895.9695693453153
892.486144118011
889.818382436037
886.1096046020588
882.6302258116858
880.7962340712547
881.0228280855549
878.378475138545
